# Análise dos registros de atendimentos da monitoria de Cálculo Diferencial e Integral 1 utilizando biblioteca pandas - Python 3 (01/2019)
## Objetivos
- Analisar os dados da monitoria do primeiro semestre de 2019;
- Familiarizar-se com o uso da biblioteca pandas.

## Questões
1. Quantos atendimentos presenciais e à distância foram realizados no semestre?
2. Qual a quantidade de atendimentos por mês?
3. Quantos alunos diferentes buscaram a monitoria?
4. Os mesmos alunos costumavam buscar a monitoria?
5. Em qual dia da semana houve maior procura?
6. Qual a duração média dos atendimentos (presencial e EAD)?
7. Qual foi o número máximo de atendimentos em um dia?
8. Quantos dias tiveram atendimento?
9. Qual a média de atendimentos por dia?

### Aparência do arquivo csv com os dados dos atendimentos
* **Código**: código do aluno;
* **Tipo**: se o atendimento foi individual ou em grupo, se foram feitos exercícios ou revisão;
* **Data**: data do atendimento;
* **Horário**: horário em que começou e terminou o atendimento;
* **Presencial**: se foi presencial (na sala de monitoria) ou EAD;
* **Dúvidas**: se foi registrado ou não as dúvidas que o aluno apresentou.

In [1]:
import pandas as pd

registers = pd.read_csv('registers_1_2019.csv')
registers.head()

,Código,Tipo,Data,Horário,Presencial,Duvidas
0,2144468,Individual,11/04/19,13:15 à 13:52,Sala,Registrado
1,2070294,Exercícios,11/04/19,13:04 à 13:37,Sala,Registrado
2,2012250,Exercícios,11/04/19,13:10 à 13:34,Sala,Registrado
3,2102544,Exercícios,11/04/19,13:25 à 13:50,Sala,Registrado
4,2102439,Exercícios,11/04/19,13:25 à 13:50,Sala,Registrado


## 1. Quantos atendimentos presenciais e à distância foram realizados no semestre?

In [2]:
total_registers = registers.index.max() + 1
presential = registers['Presencial'].isin(['Sala']).sum()
not_presential = total_registers - presential

percentage_presential = presential/total_registers*100
percentage_not_presential = not_presential/total_registers*100

list_of_quantities = [total_registers, presential, not_presential]
list_of_percentages = [100 , percentage_presential, percentage_not_presential]
list_of_indexes = ['Registers', 'Presential', 'Not Presential']

summary = pd.DataFrame({'Quantity': list_of_quantities, 'Percentage': list_of_percentages}, index = list_of_indexes)
summary[['Quantity', 'Percentage']]

,Quantity,Percentage
Registers,69,100.000000
Presential,55,79.710145
Not Presential,14,20.289855


### 2. Qual a quantidade de atendimentos por mês?

In [3]:
import datetime as dt

def return_datetime(date):
    return dt.datetime.strptime(date, '%d/%m/%y')

registers.Data = registers.Data.map(lambda d: return_datetime(d))
freq_table_months = registers.Data.map(lambda d: d.month).value_counts()
freq_table_months = freq_table_months.sort_index()
pd.DataFrame({'Quantity': list(freq_table_months)}, index = ['April', 'May', 'June', 'July'])

,Quantity
April,22
May,23
June,16
July,8


### 3. Quantos alunos diferentes buscaram a monitoria?

In [4]:
dif_students = registers.Código.unique().size
print(f'Number of different students: {dif_students}')

Number of different students: 25


### 4. Os mesmos alunos costumavam buscar a monitoria?

In [5]:
freq_table = registers.Código.value_counts().value_counts()
freq_table_percentage = freq_table/dif_students*100

freq_table_df = pd.DataFrame({'Students': freq_table, 'Percentage': freq_table_percentage})
freq_table_df = freq_table_df.rename_axis('Times helped', axis = 'index')
freq_table_df[['Students', 'Percentage']]

,Students,Percentage
Times helped,,
2,8,32.0
1,7,28.0
3,4,16.0
4,2,8.0
8,1,4.0
7,1,4.0
6,1,4.0
5,1,4.0


### 5. Em qual dia da semana houve maior procura?

In [6]:
weekday_freq = registers.Data.map(lambda d: d.weekday()).value_counts()
weekday_freq = weekday_freq.sort_index()

list_of_weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
pd.DataFrame({'Quantity': list(weekday_freq)}, index = list_of_weekdays)

,Quantity
Mon,15
Tue,6
Wed,11
Thu,17
Fri,20


### 6. Qual a duração média dos atendimentos (presencial e EAD)?

In [7]:
def return_duration(times):
    time1 = dt.datetime.strptime(times[0], '%H:%M')
    time2 = dt.datetime.strptime(times[1], '%H:%M')
    if time2 > time1:
        return time2 - time1
    else:
        return time2 - time1 + dt.timedelta(days = 1)

def sum_duration(p):
    global sum_pre, sum_ead
    global total_pre, total_ead
    if p['Presential'] == 'Sala':
        sum_pre += p['Duration']
        total_pre += 1
    else:
        sum_ead += p['Duration']
        total_ead += 1


registers.Horário = registers['Horário'].str.split(' à ')
duration = registers.Horário.map(lambda h: return_duration(h))
pre_and_dur = pd.DataFrame({'Presential': registers.Presencial, 'Duration': duration})[['Presential', 'Duration']]

sum_pre = sum_ead = dt.timedelta()
total_ead = total_pre = 0
for i in range(0, total_registers):
    sum_duration(pre_and_dur.loc[i, :])

mean_pre = sum_pre/total_pre
mean_ead = sum_ead/total_ead
pd.DataFrame({'Mean Duration': [mean_pre, mean_ead]}, index = ['Presential', 'Not Presential'])

,Mean Duration
Presential,00:31:07.636363
Not Presential,00:30:55.714285


### 7. Qual foi o número máximo de atendimentos em um dia?

In [12]:
registers_per_day = registers.groupby(['Data']).Data.count()
print(f'Maximum number of registers per day: {registers_per_day.max()}')

Maximum number of registers per day: 6


### 8. Quantos dias tiveram atendimentos?

In [10]:
freq_table_days = registers_per_day.value_counts()
total_days = freq_table_days.sum()
print(f'Days with registers: {total_days}')

Days with registers: 33


### 9. Qual a média de atendimentos por dia?

In [11]:
mean_registers = total_registers/total_days
print(f'Mean of registers per day: {mean_registers:.3} (considering only the days with registers, not all of them)')

Mean of registers per day: 2.09 (considering only the days with registers, not all of them)


In [14]:
percentage_days = freq_table_days.map(lambda f: f/total_days*100)
perc_days_df = pd.DataFrame({'Quantity of days': freq_table_days, 'Percentage': percentage_days}, index = freq_table_days.index)
perc_days_df = perc_days_df[['Quantity of days', 'Percentage']]
perc_days_df.rename_axis('Nº of registers on the day', axis = 'index')

,Quantity of days,Percentage
Nº of registers on the day,,
1,16,48.484848
2,7,21.212121
3,5,15.151515
5,2,6.060606
4,2,6.060606
6,1,3.030303


## Conclusões
Do total de 69 atendimentos, 55 (80%) foram presenciais e, 14 (20%), não presenciais.

O mês com mais registros foi Maio, e o com menos, Julho, sendo que 25 alunos diferentes buscaram monitoria durante o semestre. Desses alunos, a maioria foi atendida apenas uma ou duas vezes, sendo que o aluno mais atendido obteve 8 registros.

O dia da semana com mais atendimentos foi a sexta-feira. A duração média de um atendimento presencial foi de 31 minutos e 7 segundos, bem próximo do atendimento à distância, de 30 minutos e 55 segundos.

O número máximo de registros em um dia foi 6. Dos dias em que **houve** monitoria, 33 tiveram atendimento, com uma média de 2.09 por dia, apesar dos dias com apenas 1 serem maioria.

Com isto, foram cumpridos os objetivos.